# Initialization

In [1]:
import pymongo
import functools
import itertools
import numpy as np
import pandas as pd
from IPython.display import display, HTML

In [2]:
conn = pymongo.MongoClient('mongodb://lily.local:27017')

# Analysis

## `watchdog.exec`

Did the `watchdog.py` component successfully detect violation of the watch condition? For the Bergamot simulations, the watch event was `commit`, and the watchdog is supposed to signal a violation if 10 or more cycles elapse between consecutive `commit` events.

In [3]:
conn.nebula.watchdog.find_one().keys()

dict_keys(['_id', 'sha', 'branch', 'exec_script', 'exitcode', 'stats', 'log', 'config', 'runpath', 'pipeline', 'script', 'cmdline', 'stdout', 'stderr', 'date', 'time'])

In [4]:
print(conn.nebula.watchdog.find_one().get('log').get('0000_watchdog.py.log'))

config : {'service': 'mainmem', 'field': 'peek_latency_in_cycles', 'val': 25}
config : {'service': 'mainmem', 'field': 'filename', 'val': '/tmp/mainmem.raw'}
config : {'service': 'mainmem', 'field': 'capacity', 'val': 4294967296}
config : {'service': 'stats', 'field': 'output_filename', 'val': '/tmp/stats.json'}
config : {'service': 'stats', 'field': 'output_filename', 'val': '/home/john/src/nebula/tmp/runs/b6524326-919e-4d7f-99c5-2e2f700e5622/stats.json'}
config : {'service': 'mainmem', 'field': 'filename', 'val': '/home/john/src/nebula/tmp/runs/b6524326-919e-4d7f-99c5-2e2f700e5622/mainmem.raw'}
config : {'service': 'watchdog', 'field': 'event_name', 'val': 'commit'}
config : {'service': 'watchdog', 'field': 'event_cycles', 'val': 10}
config : {'service': 'l2', 'field': 'l2_hitlatency', 'val': 1}
config : {'service': 'mainmem', 'field': 'peek_latency_in_cycles', 'val': 10}
config : {'service': 'mainmem', 'field': 'capacity', 'val': 4294967296}
Watchdog violation!
	state.cycle         

## `quick_*.exec`

Which of these tests failed to generate a correct result for the `negate`, `sort`, `sum`, and `3sat` example binaries?

In [5]:
_correct_answer = {
    '3sat': 'solve(): [SOLUTION] s = 00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000007', # restore: ../../snapshots/3sat/mainmem.raw.000000000480000.snapshot
    'negate': '-2', # command: ../../examples/bin/negate 2
    'sort': '-3 2 5', # command: ../../examples/bin/sort 2 -3 5
    'sum': '5', # command: ../../examples/bin/sum 2 3
}

In [6]:
_benchmark = 'negate'
_quick = conn.nebula.quick.find({'cmdline': {'$regex': _benchmark}})
_incorrect = []
_correct = {}
for _q in _quick:
    _answer = _q.get('stdout').strip()
    if _correct_answer.get(_benchmark) != _answer:
        _incorrect.append(_q)
        print('{} ({}) : {} {}'.format(_answer, len(_answer), _q.get('pipeline'), _q.get('cmdline')))
        continue
    _correct.update({_q.get('pipeline'): 1 + _correct.get(_q.get('pipeline'), 0)})
print('len(_incorrect) : {}'.format(len(_incorrect)))
print('_correct        : {}'.format(_correct))

len(_incorrect) : 0
_correct        : {'pipelines/shangjuan': 64, 'pipelines/pompia': 4, 'pipelines/rangpur': 4, 'pipelines/amanatsu': 1, 'pipelines/jabara': 1}


In [7]:
_benchmark = 'sort'
_quick = conn.nebula.quick.find({'cmdline': {'$regex': _benchmark}})
_incorrect = []
_correct = {}
for _q in _quick:
    _answer = _q.get('stdout').strip()
    if _correct_answer.get(_benchmark) != _answer:
        _incorrect.append(_q)
        print('{} ({}) : {} {}'.format(_answer, len(_answer), _q.get('pipeline'), _q.get('cmdline')))
        continue
    _correct.update({_q.get('pipeline'): 1 + _correct.get(_q.get('pipeline'), 0)})
print('len(_incorrect) : {}'.format(len(_incorrect)))
print('_correct        : {}'.format(_correct))

len(_incorrect) : 0
_correct        : {'pipelines/pompia': 4, 'pipelines/shangjuan': 64, 'pipelines/rangpur': 4, 'pipelines/amanatsu': 1, 'pipelines/jabara': 1}


In [8]:
_benchmark = 'sum'
_quick = conn.nebula.quick.find({'cmdline': {'$regex': _benchmark}})
_incorrect = []
_correct = {}
for _q in _quick:
    _answer = _q.get('stdout').strip()
    if _correct_answer.get(_benchmark) != _answer:
        _incorrect.append(_q)
        print('{} ({}) : {} {}'.format(_answer, len(_answer), _q.get('pipeline'), _q.get('cmdline')))
        continue
    _correct.update({_q.get('pipeline'): 1 + _correct.get(_q.get('pipeline'), 0)})
print('len(_incorrect) : {}'.format(len(_incorrect)))
print('_correct        : {}'.format(_correct))

len(_incorrect) : 0
_correct        : {'pipelines/pompia': 4, 'pipelines/shangjuan': 64, 'pipelines/rangpur': 4, 'pipelines/amanatsu': 1, 'pipelines/jabara': 1}


In [9]:
_benchmark = '3sat'
_quick = conn.nebula.quick.find({'cmdline': {'$regex': _benchmark}})
_incorrect = []
_correct = {}
for _q in _quick:
    _answer = _q.get('stdout').strip()
    if _correct_answer.get(_benchmark) != _answer:
        _incorrect.append(_q)
        print('{} ({}) : {} {}'.format(_answer, len(_answer), _q.get('pipeline'), _q.get('cmdline')))
        continue
    _correct.update({_q.get('pipeline'): 1 + _correct.get(_q.get('pipeline'), 0)})
print('len(_incorrect) : {}'.format(len(_incorrect)))
print('_correct        : {}'.format(_correct))

len(_incorrect) : 0
_correct        : {'pipelines/shangjuan': 64, 'pipelines/pompia': 4, 'pipelines/rangpur': 4, 'pipelines/amanatsu': 1, 'pipelines/jabara': 1}


In [10]:
walk = lambda d, k: functools.reduce(lambda a, b: a.get(str(b)), k.split('.'), d)
_ipc = pd.DataFrame.from_records(sum([[{
        **walk(_q, 'config'),
        **{
            'benchmark': walk(_q, 'benchmark'),
            'pipeline': walk(_q, 'pipeline').split('/')[-1],
            'ipc': walk(_q, 'stats.0.commit.retires') / walk(_q, 'stats.cycle'),
            'cycles': walk(_q, 'stats.cycle'),
        }
}] for _q in sum(map(
        lambda b, d: list(map(
            lambda y: {**y, **{'benchmark': b}},
            filter(lambda x: x.get('stdout').strip() == _correct_answer.get(b) and '0' in x.get('stats').keys(), d)
        )),
        *zip(*[(c, conn.nebula.quick.find({'cmdline': {'$regex': c}})) for c in _correct_answer.keys()])
), [])], []))
_ipc

,fetch:l1ic_nsets,fetch:l1ic_nways,fetch:l1ic_nbytesperblock,fetch:l1ic_evictionpolicy,decode:max_bytes_to_decode,lsu:l1dc_nsets,lsu:l1dc_nways,lsu:l1dc_nbytesperblock,lsu:l1dc_evictionpolicy,l2:l2_nsets,...,mainmem:capacity,brpred:btac_entries,brpred:predictor_type,brpred:predictor_entries,alu:result_forwarding,benchmark,pipeline,ipc,cycles,decode:max_instructions_to_decode
0,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,1.0,bimodal,1.0,True,3sat,shangjuan,0.375239,26647,NaN
1,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,1.0,bimodal,64.0,True,3sat,shangjuan,0.375239,26647,NaN
2,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,1.0,gshare,1.0,True,3sat,shangjuan,0.375070,26659,NaN
3,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,1.0,gshare,64.0,True,3sat,shangjuan,0.374859,26674,NaN
4,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,64.0,bimodal,1.0,True,3sat,shangjuan,0.373613,26763,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,64.0,bimodal,64.0,False,sum,shangjuan,0.092368,23060,NaN
284,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,64.0,gshare,1.0,True,sum,shangjuan,0.093229,22847,NaN
285,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,64.0,gshare,1.0,False,sum,shangjuan,0.092005,23151,NaN
286,32,1,16,lru,16.0,16,2,16,lru,32,...,4294967296,64.0,gshare,64.0,True,sum,shangjuan,0.093058,22889,NaN


In [11]:
_ipc.set_index(['benchmark', 'pipeline']).sort_index().loc[('negate', 'shangjuan')].sort_values(by='ipc')

fetch:l1ic_nsets  fetch:l1ic_nways  \
benchmark pipeline                                        
negate    shangjuan                32                 1   
          shangjuan                32                 1   
          shangjuan                32                 1   
          shangjuan                32                 1   
          shangjuan                32                 1   
...                               ...               ...   
          shangjuan                32                 1   
          shangjuan                32                 1   
          shangjuan                32                 1   
          shangjuan                32                 1   
          shangjuan                32                 1   

                     fetch:l1ic_nbytesperblock fetch:l1ic_evictionpolicy  \
benchmark pipeline                                                         
negate    shangjuan                         16                       lru   
          shangjuan                         16                       lru   
          shangjuan                         16                       lru   
          shangjuan                         16                       lru   
          shangjuan                         16                       lru   
...                                        ...                       ...   
          shangjuan                         16                       lru   
          shangjuan                         16                       lru   
          shangjuan                         16                       lru   
          shangjuan                         16                       lru   
          shangjuan                         16                       lru   

                     decode:max_bytes_to_decode  lsu:l1dc_nsets  \
benchmark pipeline                                                
negate    shangjuan                        16.0              16   
          shangjuan                        16.0              16   
          shangjuan                        16.0              16   
          shangjuan                        16.0              16   
          shangjuan                        16.0              16   
...                                         ...             ...   
          shangjuan                        16.0              16   
          shangjuan                        16.0              16   
          shangjuan                        16.0              16   
          shangjuan                        16.0              16   
          shangjuan                        16.0              16   

                     lsu:l1dc_nways  lsu:l1dc_nbytesperblock  \
benchmark pipeline                                             
negate    shangjuan               2                       16   
          shangjuan               2                       16   
          shangjuan               2                       16   
          shangjuan               2                       16   
          shangjuan               2                       16   
...                             ...                      ...   
          shangjuan               2                       16   
          shangjuan               2                       16   
          shangjuan               2                       16   
          shangjuan               2                       16   
          shangjuan               2                       16   

                    lsu:l1dc_evictionpolicy  l2:l2_nsets  ...  \
benchmark pipeline                                        ...   
negate    shangjuan                     lru           32  ...   
          shangjuan                     lru           32  ...   
          shangjuan                     lru           32  ...   
          shangjuan                     lru           32  ...   
          shangjuan                     lru           32  ...   
...                                     ...          ...  ...   
          shangjuan                    

# Scratchpad

To query the count of pipeline model simulations currently executing, the following Bash command line is useful:

```
for p in amanatsu bergamot jabara pompia rangpur shangjuan ; do ( printf "%8d : " $( ps aux | egrep "launcher.*nebula" | egrep -v grep | egrep ${p} | wc -l ) ; echo ${p} ) ; done
```

In [12]:
assert False, 'Scratchpad area!'

AssertionError: Scratchpad area!

In [ ]:
conn.nebula.l1ic_size.delete_many({})

## `l1ic_size.exec`

Let's build a Pandas `DataFrame` of the execution cycle counts of all the L1I$ configuration runs that completed successfully.

In [ ]:
_correct_answer = '7' # command: ../../examples/bin/sum 2 3 -5 7
_tmp_df = pd.DataFrame({
    (w.get('pipeline'),) + tuple(map(lambda y: y[1], filter(lambda x: 'fetch' in x[0], w.get('config').items()))): [w.get('stats').get('cycle'), w.get('stdout').strip() == _correct_answer]
    for w in conn.nebula.l1ic_size.find({'cmdline': {'$regex': 'sum'}})
}).T.reset_index().rename(columns={
    'level_0': 'pipeline',
    'level_1': 'nsets',
    'level_2': 'nways',
    'level_3': 'nbytesperblock',
    'level_4': 'replacement',
    0: 'cycle',
    1: 'correct_answer'
}).set_index(['pipeline', 'nsets', 'nways', 'nbytesperblock', 'replacement'])
_correct_df = _tmp_df.loc[_tmp_df.correct_answer == True].sort_index()
display(HTML(_correct_df.to_html()))
print(_correct_df.describe())

        cycle correct_answer
count      48             48
unique     47              1
top     16613           True
freq        2             48


# Miscellaneous

In [ ]:
print(dir(conn))

['HOST', 'PORT', '_BaseObject__codec_options', '_BaseObject__read_concern', '_BaseObject__read_preference', '_BaseObject__write_concern', '_MongoClient__default_database_name', '_MongoClient__init_kwargs', '_MongoClient__kill_cursors_queue', '_MongoClient__lock', '_MongoClient__options', '_MongoClient__start_session', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_cleanup_cursor', '_close_cursor_now', '_close_cursor_soon', '_constructor_args', '_database_default_options', '_duplicate', '_encrypter', '_end_sessions', '_ensure_se

In [ ]:
print(conn.list_database_names())

['admin', 'config', 'local', 'nebula', 'tutorial', 'ussim']


In [ ]:
print(conn.nebula.list_collection_names())

['smoke', 'experiments']


In [ ]:
conn.nebula.experiments.find_one().keys()

dict_keys(['_id', 'sha', 'branch', 'exitcode', 'stats', 'config', 'runpath', 'pipeline', 'script', 'cmdline', 'date', 'time'])

In [ ]:
conn.nebula.experiments.find_one().get('stats').get('cycle')

2301

In [ ]:
conn.nebula.smoke.find_one().get('exitcode')

0